In [9]:
#Dependencies
from bs4 import BeautifulSoup
import requests
import json

In [10]:
import os,json
def printSoupToHtml(soup, filename='soup.html'):
    # Creating a file if not exist
    if not os.path.exists(filename):
        with open(filename, 'w', encoding='utf-8') as file:
            file.write('')
    with open(filename, 'w', encoding='utf-8') as file:
        file.write(str(soup))
    return True

def printSoupToJson(soup, filename='soup.json'):
    # Creating a file if not exist
    if not os.path.exists(filename):
        with open(filename, 'w', encoding='utf-8') as file:
            file.write('')
    with open(filename, 'w', encoding='utf-8') as file:
        file.write(json.dumps(soup))
    return True

In [11]:
#Functions
def getStringBetweenTwoWords(string, first, last):
    try:
        start = string.index(first) + len(first)
        end = string.index(last, start)
        return string[start:end]
    except ValueError:
        return ""

async def scrapeAliExpress(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    printSoupToHtml(soup,"aliexpress.html")

    # Assuming 'soup' is the BeautifulSoup object containing the parsed HTML

    # Find the <script> tag containing ACC.config.productDetails
    script_tag = soup.find('script')

    if script_tag:
        # Extract the JavaScript content from the <script> tag
        script_content = script_tag.string
        script_target_object= getStringBetweenTwoWords(script_content, 'window.runParams =', 'window.runParams.is23').replace(';', '')
        script_target_object = script_target_object.replace('data', '"data"')
        # convert the JavaScript object to a Python dictionary
        script_target_object = json.loads(script_target_object)
        
        # Extract price and title from the parsed dictionary
        price = script_target_object['data']['metaDataComponent']['ogTitle'].split('|')[0].strip()
        title = script_target_object['data']['metaDataComponent']['title'].split('|')[0].strip()
        #printSoupToJson(script_target_object['data']['imageComponent']['imagePathList'])
        ImageList = script_target_object['data']['imageComponent']['imagePathList']
        # Display the extracted price and title
        print("Price:", price)
        print("Title:", title)
        print("ImageList:", ImageList)


In [12]:

url = 'https://ar.aliexpress.com/item/1005006860843367.html?spm=a2g0o.productlist.main.1.4cbdDLvvDLvvt5&algo_pvid=7246bb87-519c-45f0-9096-37d5a5e6b81c&utparam-url=scene%3Asearch%7Cquery_from%3A'
await scrapeAliExpress(url)

Price: 50107.33EGP
Title: Apple iPhone 15 Pro ، A3104 ، APPLE ، A17 Pro ، Nano SIM ، IP68 ، إصدار CN ، علامة تجارية جديدة ونشطة ، أصلية ، منتجات أصلية
ImageList: ['https://ae01.alicdn.com/kf/S98e77c658c5b49828a14c3cdd075eb76S/Apple-iPhone-15-Pro-A3104-APPLE-A17-Pro-Nano-SIM-IP68-CN.png', 'https://ae01.alicdn.com/kf/S7fd4c9dc78df49559248bece1647335c7/Apple-iPhone-15-Pro-A3104-APPLE-A17-Pro-Nano-SIM-IP68-CN.png', 'https://ae01.alicdn.com/kf/S6a96d337ef514436ad758a123ebe190dq/Apple-iPhone-15-Pro-A3104-APPLE-A17-Pro-Nano-SIM-IP68-CN.png', 'https://ae01.alicdn.com/kf/S6c14bb6e47d343f8b389f083a8998fcf6/Apple-iPhone-15-Pro-A3104-APPLE-A17-Pro-Nano-SIM-IP68-CN.png']
